# Une fonction pour simplifier l'accès aux données

In [1]:
def extractField(s: String, fieldNumber: Int): String = {
    val fields = s.split('\t')
    if (fieldNumber >= fields.length) "" else fields(fieldNumber)
}

In [2]:
println(extractField("2	CASSIOPEE	2009	33	3.0000", 0))
println(extractField("2	CASSIOPEE	2009	33	3.0000", 1))
println(extractField("2	CASSIOPEE	2009	33	3.0000", 2))
println(extractField("2	CASSIOPEE	2009	33	3.0000", 3))
println(extractField("2	CASSIOPEE	2009	33	3.0000", 4))
println(extractField("2	CASSIOPEE	2009	33	3.0000", 5))

2
CASSIOPEE
2009
33
3.0000


# Charger les données
1. Créer le RDD `lignes` à partir du répertoire `prenoms_sample.txt`

In [3]:
val lignes = sc.textFile("prenoms_sample.txt")
lignes.take(10).foreach(println)

2	LISON	2004	69	10.0000
2	LISON	2011	59	59.0000
2	LISON	2012	86	4.0000
2	LISON	2014	90	3.0000
2	LIVIA	2002	94	4.0000
2	LIYA	2011	62	3.0000
2	LIYAH	2010	93	6.0000
2	LIZ�A	2014	76	3.0000
2	LOANE	2008	37	12.0000
2	LOLA	2006	11	23.0000


# Transformer les lignes en prénoms
1. En appliquant la méthode `map`, créer le RDD `prenoms` à partir de `lignes`

In [4]:
val prenoms = lignes.map(l => (
    extractField(l, 0).charAt(0),
    extractField(l, 1),
    extractField(l, 2).toInt,
    extractField(l, 3).toInt,
    extractField(l, 4).toDouble.toInt
))
prenoms.take(10).foreach(println)

(2,LISON,2004,69,10)
(2,LISON,2011,59,59)
(2,LISON,2012,86,4)
(2,LISON,2014,90,3)
(2,LIVIA,2002,94,4)
(2,LIYA,2011,62,3)
(2,LIYAH,2010,93,6)
(2,LIZ�A,2014,76,3)
(2,LOANE,2008,37,12)
(2,LOLA,2006,11,23)


# Interroger les données
La documentation des méthodes d'un RDD est disponible ([RDD](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD), [PairRDDFunctions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.PairRDDFunctions)).

1. Rappeler ce que sont les *transformations* et les *actions*
1. Donner, pour chaque prénom, son nombre d'occurences (`map` et `reduceByKey`)

In [1]:
//TODO

Name: Syntax Error.
Message: 
StackTrace: 

1. Donner le nombre total de naissances avec un prénom féminin (`filter`, `map`, `reduce` ou `sum`)

In [2]:
//TODO

Name: Syntax Error.
Message: 
StackTrace: 

1. Donner l'effectif maximal et minimal par prénom (`map`, `aggregateByKey`)

In [3]:
//TODO

Name: Syntax Error.
Message: 
StackTrace: 

1. Sur le modèle des prénoms, charger les données des départements
1. Donner, pour chaque nom de département, le prénom le plus fréquent depuis l'année 2000

In [4]:
//TODO

Name: Syntax Error.
Message: 
StackTrace: 

In [5]:
//TODO

Name: Syntax Error.
Message: 
StackTrace: 